In [2]:
import gluonbook as gb
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

In [4]:
(corpus_indices, char_to_idx, idx_to_char,vocab_size) = gb.load_data_jay_lyrics()

In [6]:
nd.one_hot(nd.array([1,2]),vocab_size)



[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [7]:
#每次采样的小批量的形状是（批量大小，时间步数）。下面的函数将这样的小批量变换成数个可以输入进网络的形状为（批量大小，词典大小）、
# 的矩阵，总数与时间步数相等。

def to_onehot(X, size): # 本函数已保存在gluonbook 包中⽅便以后使⽤。
    return [nd.one_hot(x, size) for x in X.T]

In [8]:
X = nd.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [9]:
X


[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
<NDArray 2x5 @cpu(0)>

In [10]:
inputs #时间步数里第一步的两个批量的单词

[
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>,
 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>]

6.4.2 初始化模型参数

In [11]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = gb.try_gpu()
print('will use', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    # 隐藏层参数。
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    # 输出层参数。
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度。
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

will use cpu(0)


6.4.3 定义模型


In [12]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [14]:
def rnn(inputs, state, params):
    W_xh,W_hh,b_h, W_hq,b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X,W_xh) + nd.dot(H,W_hh) + b_h)
        Y = nd.dot(H,W_hq) + b_q
        outputs.append(Y)
    return outputs,(H,)

In [15]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

In [21]:
# 本函数已保存在gluonbook 包中⽅便以后使⽤。
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上⼀时间步的输出作为当前时间步的输⼊。
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)

        # 计算输出和更新隐藏状态。
        (Y, state) = rnn(X, state, params)
        # 下⼀个时间步的输⼊是prefix ⾥的字符或者当前的最佳预测字符。
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [22]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
ctx, idx_to_char, char_to_idx)


'分开手戒三呢滩C成因鸣蛛'

In [23]:
norm = nd.array([0.0], ctx)

In [31]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0.0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [32]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,vocab_size, ctx, corpus_indices, idx_to_char,char_to_idx, is_random_iter, num_epochs, num_steps,lr, clipping_theta, batch_size, pred_period,pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = gb.data_iter_random
    else:
        data_iter_fn = gb.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        if not is_random_iter: # 如使⽤相邻采样，在epoch 开始时初始化隐藏状态。
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        loss_sum, start = 0.0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for t, (X, Y) in enumerate(data_iter):
            if is_random_iter: # 如使⽤随机采样，在每个⼩批量更新前初始化隐藏状态。
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else: # 否则需要使⽤detach 函数从计算图分离隐藏状态。
    
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs 有num_steps 个形状为（batch_size，vocab_size）的矩阵。
                (outputs, state) = rnn(inputs, state, params)
                # 拼接之后形状为（num_steps * batch_size，vocab_size）。
                outputs = nd.concat(*outputs, dim=0)
                # Y 的形状是（batch_size，num_steps），转置后再变成⻓度为
                # batch * num_steps 的向量，这样跟输出的⾏⼀⼀对应。
                y = Y.T.reshape((-1,))
                # 使⽤交叉熵损失计算平均分类误差。
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx) # 裁剪梯度。
            gb.sgd(params, lr, 1) # 因为误差已经取过均值，梯度不⽤再做平均。
            loss_sum += l.asscalar()
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, math.exp(loss_sum / (t + 1)), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [26]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 200, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [33]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,vocab_size, ctx, corpus_indices, idx_to_char,char_to_idx, True, num_epochs, num_steps, lr,clipping_theta, batch_size, pred_period, pred_len,prefixes)

epoch 50, perplexity 72.825915, time 0.80 sec
 - 分开 我想要你的手 它知我不多你的 我感能的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人
 - 不分开  我有你这 你的让我的狂 感能的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏
epoch 100, perplexity 10.392442, time 0.81 sec
 - 分开 一颗用双截棍的在斑鸠 印地安只 在小村外的溪边河口 默反的让我面狂的可爱女人 坏柔的让我疯狂的可爱
 - 不分开吗 我爱你 爱情我 别子就 一颗两颗三颗四颗 连成线背著背默默许的可言 我想你的爱写在西元前 深埋在
epoch 150, perplexity 2.921752, time 0.83 sec
 - 分开 一直在它心仪的母斑鸠 爱着一阵饿昏 腿短毛的里鸠 平短成的板 它一定热昏 你在那里 在小村外的溪边
 - 不分开扫 我不能再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 
epoch 200, perplexity 1.582987, time 0.85 sec
 - 分开 一直在钩拳的豆等酱 用对今断的你过得好不能 整个画面是你 想你想的睡不 得每都难我 娘子一人鸦喝 
 - 不分开扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在武我一起好  哼后用 快过了双
